In [1]:
import requests
headers = {'User-Agent': 'XY','Content-type': 'application/json'}
r = requests.get('http://rest.kegg.jp/get/H00020', headers=headers)
# print(r.status_code)
# print(r.text)

s = r.text.split("DRUG")[1].split("DBLINKS")[0]
print(s)


        Capecitabine [DR:D01223]
            Floxuridine [DR:D04197]
            Trifluridine and tipiracil hydrochloride [DR:D10526]
            Irinotecan hydrochloride [DR:D01061]
            Encorafenib [DR:D11053] (BRAF mutation positive)
            Regorafenib hydrate [DR:D10137]
            Cetuximab [DR:D03455] (K-Ras wild type, EGFR-expressing)
            Panitumumab [DR:D05350] (KRAS and NRAS wild type)
            Nivolumab [DR:D10316] (MSI-H or dMMR)
            Pembrolizumab [DR:D10574] (MSI-H)
            Bevacizumab [DR:D06409]
            Ramucirumab [DR:D09371]
            Oxaliplatin [DR:D01790]
            Aflibercept [DR:D09574]
            Leucovorin calcium [DR:D01211]
            Levoleucovorin calcium [DR:D04715]
            Calcium levofolinate hydrate [DR:D11555]
            Sodium levofolinate [DR:D11394]



In [7]:
drugnames = []; drugcodes = [] 
for i in s.split("\n"):
    if i != '':
        name = i.split(" [DR:")[0].lstrip() 
        drugnames.append(name)


        DR = i.split(" [DR:")[1].split("]")[0]
        drugcodes.append(DR)

print(drugnames)
print(drugcodes)
print(len(drugnames), len(drugcodes))

['Capecitabine', 'Floxuridine', 'Trifluridine and tipiracil hydrochloride', 'Irinotecan hydrochloride', 'Encorafenib', 'Regorafenib hydrate', 'Cetuximab', 'Panitumumab', 'Nivolumab', 'Pembrolizumab', 'Bevacizumab', 'Ramucirumab', 'Oxaliplatin', 'Aflibercept', 'Leucovorin calcium', 'Levoleucovorin calcium', 'Calcium levofolinate hydrate', 'Sodium levofolinate']
['D01223', 'D04197', 'D10526', 'D01061', 'D11053', 'D10137', 'D03455', 'D05350', 'D10316', 'D10574', 'D06409', 'D09371', 'D01790', 'D09574', 'D01211', 'D04715', 'D11555', 'D11394']
18 18


In [8]:
import re
import itertools
splitters = ["METABOLISM", "INTERACTION", "SYN_MAP", "STR_MAP", "OTHER_MAP", "BRITE", "DBLINKS"]

drug_linkers = []; error_indices = [] 
drug_target_genes = []; drug_target_pathways = [] 
for index, dc in enumerate(drugcodes): 

    r = requests.get('http://rest.kegg.jp/get/'+dc, headers=headers)
    try:
        target_gene = r.text.split("TARGET      ")[1].split("  ")[0].split('\n')[0]  # did check that only 1 target gene exists for all 
        target_pathway = r.text.split("  PATHWAY   ")[1]
    
        the_newest_regex = "({})".format("|".join(re.escape(s) for s in splitters))
        the_raw_split = re.split(the_newest_regex, target_pathway)
        the_actual_split = ["".join(x) for x in itertools.zip_longest([""] + the_raw_split[1::2], the_raw_split[::2], fillvalue='')]

        target_pathway = [] 
        for i in the_actual_split[0].split("\n"):
            target_pathway.append(i.lstrip())
        target_pathway = target_pathway[:-1]

        drug_target_pathways.append(target_pathway)
        drug_target_genes.append(target_gene)
#         drug_target_gene_codes.append("HSA:" + target_gene.split(" [HSA:")[1].split("]")[0])
        
    except:
        error_indices.append(index)

for i in sorted(error_indices, reverse=True):
    del drugnames[i]
    del drugcodes[i]
print(len(drugnames))
print(len(drugcodes))      
print(len(drug_target_pathways))
print(len(drug_target_genes))
# print(len(drug_target_gene_codes))


11
11
11
11


In [10]:
print(drug_target_genes)

['TYMS [HSA:7298] [KO:K00560]', 'TYMS [HSA:7298] [KO:K00560]', 'BRAF* [HSA_VAR:673v1] [HSA:673] [KO:K04365]', 'VEGFR1 (FLT1) [HSA:2321] [KO:K05096]', 'EGFR [HSA:1956] [KO:K04361]', 'EGFR [HSA:1956] [KO:K04361]', 'PDCD1 (PD1, CD279) [HSA:5133] [KO:K06744]', 'PDCD1 (PD1, CD279) [HSA:5133] [KO:K06744]', 'VEGFA [HSA:7422] [KO:K05448]', 'VEGFR2 (KDR) [HSA:3791] [KO:K05098]', 'VEGFA [HSA:7422] [KO:K05448]']


In [11]:
print(drug_target_pathways)

[['hsa00240(7298)  Pyrimidine metabolism', 'hsa00670(7298)  One carbon pool by folate'], ['hsa00240(7298)  Pyrimidine metabolism', 'hsa00670(7298)  One carbon pool by folate'], ['hsa04010(673)  MAPK signaling pathway', 'hsa04012(673)  ErbB signaling pathway', 'hsa05200(673)  Pathways in cancer', 'hsa05218(673)  Melanoma'], ['hsa04010(2260+2321+2324+3791+3815+5159+5894+673+7010)  MAPK signaling pathway', 'hsa04370(3791+5894)  VEGF signaling pathway', 'hsa05200(2260+2324+3815+5159+5894+5979+673)  Pathways in cancer', 'hsa05210(5894+673)  Colorectal cancer', 'hsa05225(5894+673)  Hepatocellular carcinoma'], ['hsa04010(1956)  MAPK signaling pathway', 'hsa04012(1956)  ErbB signaling pathway', 'hsa05200(1956)  Pathways in cancer'], ['hsa04010(1956)  MAPK signaling pathway', 'hsa04012(1956)  ErbB signaling pathway', 'hsa05200(1956)  Pathways in cancer'], ['hsa04514(5133)  Cell adhesion molecules', 'hsa04660(5133)  T cell receptor signaling pathway', 'hsa05235(5133)  PD-L1 expression and PD-1 c